In [6]:
#!/usr/bin/env python
# -*- coding: iso-8859-1 -*-
import sqlite3
from datetime import datetime,date,timedelta
import calendar
import tkinter as tk
from tkinter import ttk
from tkinter.messagebox import askokcancel, showerror, showinfo, WARNING
from functools import partial
import random
from copy import deepcopy
import locale

### Set up database, if not already done ###
verbindung = sqlite3.connect("habit-tracker.db")
zeiger = verbindung.cursor()
zeiger2 = verbindung.cursor()
sql_anweisung = """
CREATE TABLE IF NOT EXISTS `habits` (
  `pk` INTEGER PRIMARY KEY AUTOINCREMENT,
  `datecreate` DATE,
  `startdate` DATE,
  `enddate` DATE,
  `name` VARCHAR(100),
  `periode` CHAR(1));"""
zeiger.execute(sql_anweisung)
verbindung.commit()
sql_anweisung = """
CREATE TABLE IF NOT EXISTS `checked` (
  `pk` INTEGER PRIMARY KEY AUTOINCREMENT,
  `checkdate` DATE,
  `checked` BOOLEAN,
  `habitID` INTEGER,
  FOREIGN KEY(habitID) REFERENCES habits(pk));"""
zeiger.execute(sql_anweisung)
verbindung.commit()
verbindung.close()

# Define global variables
habit_objects={}
habit_shows={}

class Habit():
    """ Creates the objects Habit for our Habittracker."""
    counter=0
    optionmenuValue='All'
    selected_row=0
    selected_info=0
    def __init__(self,pk,datecreate,startdate,enddate,name,periode):
        """Initializing a new habit object"""
        self.pk=pk
        self.datecreate=datecreate
        self.startdate=startdate
        self.enddate=enddate
        self.name=name
        self.periode=periode
        Habit.counter += 1
        
    @staticmethod
    def new(startdate,enddate,name,periode):
        global habit_objects
        global habit_shows
        """Creates a new habit object also in the database. The PK (primary key) created in the database is then read from the DB 
        and written into the object. The values datecreate and checked are entered automatically.
        """
        currenttime=datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
        connection = sqlite3.connect("habit-tracker.db")
        pointer = connection.cursor()
        pointer.execute("""
        INSERT INTO habits (
        datecreate,startdate,enddate,name,periode)
        VALUES (?,?,?,?,?) RETURNING pk""",
        (currenttime,startdate,enddate,name,periode))
        content = pointer.fetchall()
        pk_object=content[0][0]
        pointer.execute("""
        INSERT INTO checked (
        checkdate,checked,habitID)
        VALUES (?,?,?)""",
        ((startdate,False,pk_object)
        ))
        connection.commit()
        connection.close()
        habit_objects[Habit.counter]=Habit(pk_object,currenttime,startdate,enddate,name,periode)
        # Now check if new habit has to be entered into the habit_shows dict (depending on the filter)
        if Habit.optionmenuValue=='All': # habit_objects is only copied
            habit_shows = deepcopy(habit_objects)
        elif Habit.optionmenuValue=='weekly' and periode=='w': # Only if the filter is selected weekly, the weekly habit is entered.
            habit_shows[len(habit_shows)+1]=habit_objects[Habit.counter]
        elif Habit.optionmenuValue=='daily' and periode=='d':  # Only if the daily filter is selected, the daily habit is entered.
            habit_shows[len(habit_shows)+1]=habit_objects[Habit.counter]        
        return pk_object

    def delete(self,shows_row,object_row):
        """ Deletes a habit object from the database and from the habit_objects and habit_shows list. """
        connection = sqlite3.connect("habit-tracker.db")
        pointer = connection.cursor()
        pointer.execute("DELETE FROM checked WHERE habitID='"+str(self.pk)+"'")
        connection.commit()
        pointer.execute("DELETE FROM habits WHERE pk='"+str(self.pk)+"'")
        connection.commit()
        connection.close()
        del habit_shows[shows_row]
        del habit_objects[object_row]
        #Jetzt neu durchnummerieren
        habit_temp = deepcopy(habit_objects)
       
        habit_objects.clear()
        n=1
        for key,value in habit_temp.items():
            habit_objects[n]=value
            n+=1
        habit_temp.clear()
        habit_temp = deepcopy(habit_shows)
        habit_shows.clear()
        n=1
        for key,value in habit_temp.items():
            habit_shows[n]=value
            n+=1
        app.clear_frame(StartPage)

        Habit.counter -= 1
        
    def update(self,name_val,startdate_val,enddate_val,periode_val):
        """ Update of an existing habit. The database entries are changed as well as the values of the habit class of the habit object. 
            The filter_temporally function is called to correct the habit_shows list if the user has changed the period of the habit."""
        self.startdate = startdate_val
        self.enddate = enddate_val
        self.name = name_val
        self.periode = periode_val
        connection = sqlite3.connect("habit-tracker.db")
        pointer = connection.cursor()
        sql_statement = "UPDATE habits SET startdate='%s',enddate='%s',name='%s',periode='%s' WHERE pk='%d';" % (startdate_val,enddate_val,name_val,periode_val,self.pk)
        pointer.execute(sql_statement)
        connection.commit()
        connection.close()
        #app.filter_temporally(Habit.optionmenuValue)
        app.clear_frame(StartPage)
        
    def save_check(self):
        """This function saves the current Done date in the database.
           It is checked if an entry already exists, if this is the case, the entry is deleted.
           The function is called when the user clicks on the checkbox to complete his habit."""
        connection = sqlite3.connect("habit-tracker.db")
        pointer = connection.cursor()
        pointer.execute("SELECT MAX(checkdate),checked,pk FROM checked WHERE habitID='"+str(self.pk)+"'")
        content = pointer.fetchall()
    
        date_object_Habit_start = datetime.strptime(self.startdate, '%Y-%m-%d').date()
        date_object_Habit_end = datetime.strptime(self.enddate, '%Y-%m-%d').date()
        today_date = date.today()
        if str(content[0][0])!='None': # Are there any checked values in the database for this habit at all? Yes
            date_object_database = datetime.strptime(content[0][0], '%Y-%m-%d').date()
            bool_object_database = bool(content[0][1])
            checked_pk=int(content[0][2])
        else:                          #No: No check values in the DB: Create a first value yourself
            if self.periode=='w':
                if date_object_Habit_end>=today_date: 
                    date_object_database=date_object_Habit_start+timedelta(6)
                else:
                    date_object_database=date_object_Habit_end
            else:
                    date_object_database=date_object_Habit_start
            checked_pk=0    
            bool_object_database = False

        if self.periode=='d':                      # Period = daily 
            if date_object_database==today_date: 
                if checked_pk>0:
                    if bool_object_database==False: # Current date present in the database value but not yet checked: Set to True
                        pointer.execute("UPDATE checked SET checked='1' WHERE pk='"+str(checked_pk)+"'")
                        connection.commit()                         
                    else:                           # Current database value exists, will be deleted
                        pointer.execute("DELETE FROM checked WHERE pk='"+str(checked_pk)+"'")
                        connection.commit()
                else:
                    pointer.execute("""
                    INSERT INTO checked (
                    checkdate,checked,habitID)
                    VALUES (?,?,?)""",
                    ((today_date.strftime('%Y-%m-%d'),True,self.pk)
                    ))
                    connection.commit()


            else:                                # No database value, current date is written to DB.
                pointer.execute("""
                INSERT INTO checked (
                checkdate,checked,habitID)
                VALUES (?,?,?)""",
                ((today_date.strftime('%Y-%m-%d'),True,self.pk)
                ))
                connection.commit()

        else:                                      # Period = weekly
            delta=today_date-date_object_Habit_start
            weeks_next=int(delta.days/7)+1
            sumdays_next=weeks_next*7-1
            weeks_prev=int(delta.days/7)
            sumdays_prev=weeks_prev*7
            to_date=date_object_Habit_start+timedelta(sumdays_next)
            from_date=date_object_Habit_start+timedelta(sumdays_prev)
            if date_object_database>=from_date and date_object_database<=to_date: # Current database value exists, will be deleted, user has deactivated checkbox.
                if checked_pk>0:
                    pointer.execute("DELETE FROM checked WHERE pk='"+str(checked_pk)+"'")
                    connection.commit()
                else:
                    pointer.execute("""
                    INSERT INTO checked (
                    checkdate,checked,habitID)
                    VALUES (?,?,?)""",
                    ((today_date.strftime('%Y-%m-%d'),True,self.pk)
                    ))
                    connection.commit()
            else:                                                                 # No database value, current date is written to DB.
                pointer.execute("""
                INSERT INTO checked (
                checkdate,checked,habitID)
                VALUES (?,?,?)""",
                ((today_date.strftime('%Y-%m-%d'),True,self.pk)
                ))
                connection.commit()
        pointer.execute("SELECT * FROM checked;")
        inhalt = pointer.fetchall()

        connection.close() 

    def last_check_value(self):
        """Gibt den letzten Check Wert zurück """
        connection = sqlite3.connect("habit-tracker.db")
        pointer = connection.cursor()
        pointer.execute("SELECT MAX(checkdate),checked FROM checked WHERE habitID='"+str(self.pk)+"'")
        content = pointer.fetchall()
        connection.close()
        date_object_Habit_start = datetime.strptime(str(self.startdate), '%Y-%m-%d').date()
        date_object_Habit_end = datetime.strptime(str(self.enddate), '%Y-%m-%d').date()
        today_date = date.today()
        if str(content[0][0])!='None': # Are there any checked values in the database for this habit at all? Yes
            date_object_database = datetime.strptime(content[0][0], '%Y-%m-%d').date() #Get date from DB (most recent date)
            bool_object_database = bool(content[0][1])                                 #Get check value
        else: #No: No values in DB: Create first value yourself
            if date_object_Habit_end<today_date: 
                return [date_object_Habit_end.strftime('%d.%m.%Y'),False,True]     # End date before today: Return end date and False, disable checkbox
            elif date_object_Habit_end==today_date:
                return [date_object_Habit_end.strftime('%d.%m.%Y'),False,False]    # End date today: Return end date and False, enable checkbox
            else:                                                                  # From here on everything normal: End date not yet reached
                date_object_database=date_object_Habit_start                       # Pretends that we have an initial value with the start date
                bool_object_database = False                                       # Is False, had nothing in the DB yet
        if self.periode=='w': #Weekly period
            # Now calculate last day of last week
            delta=today_date-date_object_Habit_start
            weeks_today_start=int(delta.days/7)+1 # Weeks until today
            delta=date_object_database-date_object_Habit_start
            weeks_dbase_start=int(delta.days/7)+1 # Weeks until today
            to_date=date_object_Habit_start+timedelta(weeks_today_start*7-1)
            if weeks_dbase_start==weeks_today_start: # Database value is in the current week
                return [to_date.strftime('%d.%m.%Y'),bool_object_database,False]
            else:                                    # Completed date not in current week
                return [to_date.strftime('%d.%m.%Y'),False,False]
        else: # Daily period
            if date_object_database>today_date:               # Start date in the future
                return [date_object_database.strftime('%d.%m.%Y'),bool_object_database,True]
            elif date_object_Habit_end<today_date:            # If end date past
                if date_object_database==date_object_Habit_end:    #Day exactly completed
                    return [date_object_database.strftime('%d.%m.%Y'),bool_object_database,False] # Values [Bis-erledigen-Datum, Checked, Disable Checkbox]
                else: #content[0][0]<self.enddate: # If last check before end date and not today, then Check = False
                    return [date_object_database.strftime('%d.%m.%Y'),False,False]
            elif date_object_Habit_end==today_date:         # End date is today
                if date_object_database==date_object_Habit_end:    # Check is done today, then return DB entry, check can still change
                    return [date_object_database.strftime('%d.%m.%Y'),bool_object_database,False]
                else:                              # End date not today: Then past, cannot be changed: False
                    return [today_date.strftime('%d.%m.%Y'),False,True]
            else:                                  # End date in the future
                if date_object_database<today_date:       # Last entry past: Output date today and False.                    
                    return [today_date.strftime('%d.%m.%Y'),False,False]
                else:                              # Entry today, returns DB values directly.
                    return [date_object_database.strftime('%d.%m.%Y'),bool_object_database,False]
    
    def series_check_values(self):
        """Adds dates to the checked table of the database if they are in the past. After that
           better use the DB to filter and calculate the longest data series, since we want to output both positive and negative series.
           The determination of the series, is done directly on the database."""
        connection = sqlite3.connect("habit-tracker.db")
        pointer = connection.cursor()
        pointer.execute("SELECT checkdate,checked FROM checked WHERE habitID='"+str(self.pk)+"' ORDER BY checkdate")
        content = pointer.fetchall()
        date_object_Habit_start = datetime.strptime(self.startdate, '%Y-%m-%d').date()
        date_object_Habit_end = datetime.strptime(self.enddate, '%Y-%m-%d').date()
        today_date = date.today()

        if self.periode=='d':                      # Period = daily  
            if date_object_Habit_start<=today_date: #Start date not in the future, in which case we return per else ''. 
                from_date=date_object_Habit_start
                if date_object_Habit_end<today_date: 
                    to_date=date_object_Habit_end
                else:
                    to_date=today_date
                delta = to_date-from_date
                """We now count up one tag at a time, and compare whether the tag is present in the content list from the DB."""
                checked_counter=0
                checked_length=len(content)-1
                for z in range(0,delta.days+1):
                    if date_object_Habit_start+timedelta(z)==datetime.strptime(content[checked_counter][0], '%Y-%m-%d').date():
                        if checked_counter<checked_length: checked_counter+=1
                    else:
                        pointer.execute("""
                        INSERT INTO checked (
                        checkdate,checked,habitID)
                        VALUES (?,?,?)""",
                        (((date_object_Habit_start+timedelta(z)).strftime('%Y-%m-%d'),False,self.pk)
                        ))
                # We calculate directly on the database the longest series
                # Longest positive series
                pointer.execute("WITH checked_groups AS (SELECT checkdate, julianday(checkdate) - julianday('"+self.startdate+"') - ROW_NUMBER() OVER (ORDER BY checkdate) AS grp FROM checked WHERE habitID='"+str(self.pk)+"' AND checked=True) SELECT MIN(checkdate), MAX(checkdate), MIN(julianday(checkdate)), MAX(julianday(checkdate)), MAX(julianday(checkdate)) - MIN(julianday(checkdate)) + 1 AS length FROM checked_groups GROUP BY grp ORDER BY length DESC")
                content = pointer.fetchall()
                if len(content)==0:
                    return_value=[['','','','d']]
                else:
                    return_value=[[content[0][0],content[0][1],int(content[0][4]),'d']]
                #Longest negative series
                pointer.execute("WITH checked_groups AS (SELECT checkdate, julianday(checkdate) - julianday('"+self.startdate+"') - ROW_NUMBER() OVER (ORDER BY checkdate) AS grp FROM checked WHERE habitID='"+str(self.pk)+"' AND checked=False) SELECT MIN(checkdate), MAX(checkdate), MIN(julianday(checkdate)), MAX(julianday(checkdate)), MAX(julianday(checkdate)) - MIN(julianday(checkdate)) + 1 AS length FROM checked_groups GROUP BY grp ORDER BY length DESC")
                content = pointer.fetchall()
                return_value.append([content[0][0],content[0][1],int(content[0][4])])
                connection.commit()
                connection.close()
                return return_value
            else:
                return [['','','','d'],['','','','d']]
        else:                            # Period = weekly
            if date_object_Habit_start<today_date: # Start date not in the future, in which case we return per else ''. 
                if date_object_Habit_end<today_date:   # If end date in the past, we look only to end date
                    to_date=date_object_Habit_end
                else:
                    to_date=today_date                 # Can only remain the same to today: we take that in with us
                if date_object_Habit_start<today_date: # The start date is in the past, so normal case, from start date
                    from_date=date_object_Habit_start
                delta = to_date-from_date
                weeks=delta.days/7
                checked_counter=0
                checked_length=len(content)-1
                for z in range(0,int(weeks+1)):
                    db_delta=((datetime.strptime(content[checked_counter][0], '%Y-%m-%d').date()-date_object_Habit_start).days)/7 #kkk
                    if int(z)==int(db_delta):
                        if checked_counter<checked_length: checked_counter+=1
                    else:
                        pointer.execute("""
                        INSERT INTO checked (
                        checkdate,checked,habitID)
                        VALUES (?,?,?)""",
                        (((date_object_Habit_start+timedelta(z*7)).strftime('%Y-%m-%d'),False,self.pk)
                        ))
                        connection.commit()
                # We calculate directly on the database the longest series
                # Longest positive series
                pointer.execute("WITH checked_groups AS (SELECT checkdate, (julianday(checkdate) - julianday('"+self.startdate+"'))/7 - ROW_NUMBER() OVER (ORDER BY checkdate) AS grp FROM checked WHERE habitID='"+str(self.pk)+"' AND checked=True) SELECT MIN(checkdate), MAX(checkdate), MIN(julianday(checkdate)), MAX(julianday(checkdate)), (MAX(julianday(checkdate)) - MIN(julianday(checkdate)))/7 + 1 AS length FROM checked_groups GROUP BY grp ORDER BY length DESC")
                content = pointer.fetchall()
                if len(content)>0:
                    return_value=[[content[0][0],(datetime.strptime(content[0][1], '%Y-%m-%d').date()+timedelta(6)).strftime('%Y-%m-%d'),int(content[0][4]),'w']]
                else:
                    return_value=[['','','','w']]
                #Längste Negativ Serie
                pointer.execute("WITH checked_groups AS (SELECT checkdate, julianday(checkdate) - julianday('"+self.startdate+"') - ROW_NUMBER() OVER (ORDER BY checkdate) AS grp FROM checked WHERE habitID='"+str(self.pk)+"' AND checked=False) SELECT MIN(checkdate), MAX(checkdate), MIN(julianday(checkdate)), MAX(julianday(checkdate)), MAX(julianday(checkdate)) - MIN(julianday(checkdate)) + 1 AS length FROM checked_groups GROUP BY grp ORDER BY length DESC")
                content = pointer.fetchall()
                if len(content)>0:
                    return_value.append([content[0][0],(datetime.strptime(content[0][1], '%Y-%m-%d').date()+timedelta(6)).strftime('%Y-%m-%d'),int(content[0][4]),'w'])
                else:
                    return_value.append(['','','','w'])
                connection.commit()
                connection.close()
                return return_value
            else:
                return [['','','','w'],['','','','w']]
        connection.commit()
        connection.close()

class Table(tk.Frame):
    """Lists the Habits in a table for our start page."""

    def __init__(self, controller, rows=len(habit_shows), columns=7, bg="#b0bbce"):
        super().__init__(controller, padx=1, pady=1, bg='gray')
        def edit_habit(n):
            app.import_values(PageNewEdit,n)
            app.show_frame(PageNewEdit,n)
        def info_habit(n): #nnn
            app.import_info(PageInfo,n)
            app.show_frame(PageInfo,n)

        self.zeitlich = ("All","daily","weekly")
        self.option_var_zeitlich = tk.StringVar(self)
        self.checkbutton_state = tk.BooleanVar(self)
        self.checkbutton_selection = tk.IntVar(self)
        self.cells = []
        cell_row = []
        cell = tk.Label(self, text='Habit', bg="#e0e0e0")
        cell.config(padx=4, pady=4, width=50, anchor='w')
        cell.grid(row=1, column=0, padx=1, pady=1, sticky='w')
        cell_row.append(cell)

        cell = ttk.OptionMenu(self,self.option_var_zeitlich,Habit.optionmenuValue,*self.zeitlich, command=lambda val=self.option_var_zeitlich.get(): app.filter_temporally(val))
        cell.config(width=11)
        cell.grid(row=1, column=1, padx=1, pady=1)
        cell_row.append(cell)
        cell = tk.Label(self, text='Erledigt', bg="#ffd0d0")
        cell.config(padx=4, pady=4, width=6)
        cell.grid(row=1, column=2, padx=1, pady=1, sticky=tk.E)
        cell_row.append(cell)

        cell = tk.Label(self, text='Bis', bg="#e0e0e0")
        cell.config(padx=4, pady=4, width=11)
        cell.grid(row=1, column=3, padx=1, pady=1, sticky=tk.E)
        cell_row.append(cell)

        cell = tk.Label(self, bg="#e0e0e0")
        cell.config(padx=4, pady=4, width=2,)
        cell.grid(row=1, column=4, padx=1, pady=1, sticky=tk.E)
        cell_row.append(cell)

        cell = tk.Label(self, text='', bg="#e0e0e0")
        cell.config(padx=4, pady=4, width=2)
        cell.grid(row=1, column=5, padx=1, pady=1)
        cell_row.append(cell)

        cell = tk.Label(self, text='', bg="#e0e0e0")
        cell.config(padx=4, pady=4, width=2)
        cell.grid(row=1, column=6, padx=1, pady=1)
        cell_row.append(cell)

        self.cells.append(cell_row)
        self.checkbutton_vars = []
        for row in range(1,len(habit_shows)+1):
            if habit_shows[row].periode=='w':
                text_periode='weekly'
            else:
                text_periode='daily'
            cell_row = []
            cell = tk.Label(self, text=habit_shows[row].name)
            cell.config(padx=4, pady=4, width=50, anchor='w')
            cell.grid(row=row+1, column=0, padx=1, pady=1, sticky='w')
            cell_row.append(cell)

            cell = tk.Label(self, text=text_periode)
            cell.config(padx=4, pady=4, width=12)
            cell.grid(row=row+1, column=1, padx=1, pady=1)
            cell_row.append(cell)

            cell = tk.Label(self, bg="#f0f0f0")
            cell.config(padx=4, pady=4, width=6)
            cell.grid(row=row+1, column=2, padx=1, pady=1)
            cell_row.append(cell)

            list_of_date_and_check=habit_shows[row].last_check_value()
            checkbutton_var = tk.IntVar()
            cell = ttk.Checkbutton(self, variable=checkbutton_var, command=partial(self.checkbutton_selection_func, row))
            cell.grid(column=2, row=row+1, padx=0, pady=0)
            if list_of_date_and_check[2]==True:
                cell.config(state=DISABLED)
            self.checkbutton_vars.append(checkbutton_var)

            if habit_shows[row].last_check_value()[1]==True:
                checkbutton_var.set(True)
           
            cell = tk.Label(self, text=list_of_date_and_check[0]) # Done Output to date
            cell.config(padx=4, pady=4, width=11)
            cell.grid(row=row+1, column=3, padx=1, pady=1)
            cell_row.append(cell)

            # Button Info Record
            cell = tk.Button(self, text=' I ', command=partial(info_habit, row))
            cell.grid(row=row+1, column=4, padx=1, pady=1)
            cell_row.append(cell)

            cell = tk.Button(self, text='E', command=partial(edit_habit, row)) # Edit button, the position of the habit in the habit_shows is passed with row
            cell.grid(row=row+1, column=5, padx=1, pady=1)
            cell_row.append(cell)

            cell = tk.Button(self, text='D', command=partial(self.delete, row)) # Delete button, the position of the habit in the habit_shows is passed with row
            cell.grid(row=row+1, column=6, padx=1, pady=1)
            cell_row.append(cell)

            self.cells.append(cell_row)

    def state(self):
        return map((lambda var: var.get()), self.vars)

    def checkbutton_selection_func(self,n):
        habit_shows[n].save_check()

    def delete(self,n):
        answer = askokcancel(
            title='Delete Habit!',
            message='Deletes the habit '+habit_shows[n].name,
            icon=WARNING)
        if answer:
            pk=habit_shows[n].pk
            for element in habit_objects:
                if pk==habit_objects[element].pk:
                    habit_shows[n].delete(n,element)
                    break

#Check if the database is filled
verbindung = sqlite3.connect("habit-tracker.db")
zeiger = verbindung.cursor()
zeiger.execute("SELECT COUNT(*) FROM habits;")
count_db=zeiger.fetchall()[0][0]      
verbindung.close()
        
if count_db==0:
    """When the database is empty, the test data is created, for this purpose the days are calculated, starting from today's date, according to the definition in the habit_testlist,
       habit_testlist = [minus days=start date,plus days=end date,habit name,w=weekly t=daily].
       All date entries for the 'hooks' are created and the 'hooks' are set randomly (TRUE or FALSE)."""
    habit_testlist=[[90,275,'Fitness training','w'],[60,120,'Do something with daughter','w'],[60,120,'Snacking forbidden','d'],[90,275,'Alcohol ban','d'],[30,60,'Take vitamin D','d']]
    today_date = date.today()
    for testvalue in habit_testlist:
        pk_object=Habit.new(str(today_date-timedelta(testvalue[0])),str(today_date+timedelta(testvalue[1])),testvalue[2],testvalue[3])
        if testvalue[3]=='w':
            sumdates=int(testvalue[0]/7)
            adddays=7
        else:
            sumdates=testvalue[0]
            adddays=1

        verbindung = sqlite3.connect("habit-tracker.db")
        zeiger = verbindung.cursor()
        z=0

        for i in range(sumdates): #Only the 1s are entered, i.e. only the appointments actually completed
            random_val=random.randint(0,1)
            if random_val==1:
                zeiger.execute("""
                INSERT INTO checked (
                checkdate,checked,habitID)
                VALUES (?,?,?)""",
                ((today_date-timedelta(testvalue[0]))+timedelta(z),random_val,pk_object)
                )
            z+=adddays
        verbindung.commit()
        verbindung.close()
            
else:
    """If there are entries in the database, they are imported into our Habit class."""
    connection = sqlite3.connect("habit-tracker.db")
    pointer = connection.cursor()
    pointer.execute("SELECT * FROM habits;")
    content=pointer.fetchall()     
    connection.close()
    for attributes in content:
        habit_objects[Habit.counter]=Habit(attributes[0],attributes[1],attributes[2],attributes[3],attributes[4],attributes[5])

#To display only the filtered elements when the filter is set, habit_shows is used, when filtering, only valid records are entered.
#After the start of the program all elements are displayed, therefore the list is copied.
habit_shows = deepcopy(habit_objects) 
    
class HabitMain(tk.Tk):
    """ Main program for the GUI"""
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        self.title('Habit Tracker')
        self.resizable(1, 1)
        container = tk.Frame(self)

        container.pack(side="top", fill="both", expand = True)

        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}

        for F in (StartPage, PageNewEdit, PageInfo):

            frame = F(container, self)

            self.frames[F] = frame

            frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame(StartPage)

    def show_frame(self, cont, *args):
        frame = self.frames[cont]
        frame.tkraise()

    def clear_frame(self, cont, **kwargs):
        frame = self.frames[cont]
        for widgets in frame.winfo_children():
            widgets.destroy()
        frame.tkraise()
        frame.create_widgets(cont)
    def filter_temporally(self,val):
        global habit_shows
        Habit.optionmenuValue=val
        if Habit.optionmenuValue[0]=='A':
            habit_shows = deepcopy(habit_objects)
        else:
            habit_shows.clear()
            x=1
            for element in habit_objects:
                if habit_objects[element].periode==Habit.optionmenuValue[0]:
                    habit_shows[x]=habit_objects[element]
                    x+=1
        
        app.clear_frame(StartPage)
    def import_values(self,cont,n):
        frame = self.frames[cont]
        frame.import_values(n)
    def import_info(self,cont,n):
        frame = self.frames[cont]
        frame.import_info(n)
    def new_habit(self,cont,n):
        app.import_values(PageNewEdit,n)
        app.show_frame(PageNewEdit,n)

class StartPage(tk.Frame):
    """ Start page """
    def __init__(self, parent, controller, **kwargs):
        tk.Frame.__init__(self,parent)
        # configure the grid
        self.columnconfigure(0, weight=1)
        self.columnconfigure(1, weight=4)
        self.columnconfigure(2, weight=1)
        self.create_widgets(controller)
 
    def create_widgets(self,controller):

        # Titelzeile
        start_labelTitel = ttk.Label(self, text="My Habit Tracker",font=("Verdana", 14))
        start_labelTitel.grid(column=1, row=0, sticky=tk.W, padx=5, pady=5)

        if len(habit_shows)>0:
            self.table = Table(self)
            self.table.grid(column=1, row=1, padx=5, pady=5, sticky=tk.N)
            start_labelTitel = ttk.Label(self, text="I = Info   E = Edit   D = Delete",font=("Verdana", 10))
            start_labelTitel.grid(column=1, row=2, padx=5, pady=5)

            
        else:
            start_labelTitel = ttk.Label(self, text="Please create your personal habits via the New-Button!",font=("Verdana", 10))
            start_labelTitel.grid(column=1, row=1, padx=5, pady=5)

            
        # Neu Button
        new_button = tk.Button(self, text='New', command=lambda: app.new_habit(PageNewEdit,0)) #lambda: app.show_frame(PageNewEdit)
        new_button.grid(column=1, row=2, sticky=tk.NE, padx=5, pady=5)

class PageNewEdit(tk.Frame):
    ''' Creating and editing a new habit'''
    def __init__(self, parent, controller,**kwargs):

        tk.Frame.__init__(self,parent)
        # configure the grid
        self.columnconfigure(0, weight=1)
        self.columnconfigure(1, weight=4)
        self.columnconfigure(2, weight=2)
        self.columnconfigure(3, weight=2)
        self.columnconfigure(4, weight=2)
        self.columnconfigure(5, weight=1)
        self.zeitlich = ("daily","weekly")
        self.zeitlich_kurz = ("d","w")
        self.option_var_zeitlich = tk.StringVar(self)
        self.entryText_var = tk.StringVar(self)
        self.entryDatumVon_var = tk.StringVar(self)
        self.entryDatumBis_var = tk.StringVar(self)
        self.datensatz = ['']*4
        
        self.create_widgets(controller)

    def create_widgets(self,controller):

        fontsizeEntryText=7
        # Titelzeile
        start_labelTitel = ttk.Label(self, text="Edit my habit",font=("Verdana", 14))
        start_labelTitel.grid(column=1, row=0, sticky=tk.W, padx=5, pady=5, columnspan=4)
        
        # Entry Hinweis Felder
        labelText = ttk.Label(self, text="Text",font=("Verdana", fontsizeEntryText))
        labelText.grid(column=1, row=1, sticky=tk.SW, padx=5, pady=0)

        labelDatumVon = ttk.Label(self, text="Start date",font=("Verdana", fontsizeEntryText))
        labelDatumVon.grid(column=2, row=1, sticky=tk.SW, padx=5, pady=0)

        labelDatumBis = ttk.Label(self, text="End date",font=("Verdana", fontsizeEntryText))
        labelDatumBis.grid(column=3, row=1, sticky=tk.SW, padx=5, pady=0)

        # Entrys
        entryText = ttk.Entry(self,textvariable = self.entryText_var,width=60)
        entryText.grid(column=1, row=2, sticky=tk.NW, padx=5, pady=2)
        entryText.bind('<FocusOut>',self.entry_text_in_datensatz)
        
        entryDatumVon = ttk.Entry(self,textvariable = self.entryDatumVon_var,width=10)
        entryDatumVon.grid(column=2, row=2, sticky=tk.NW, padx=5, pady=2)
        entryDatumVon.bind('<FocusOut>',self.entry_DatumVon_in_datensatz)

        entryDatumBis = ttk.Entry(self,textvariable = self.entryDatumBis_var,width=10)
        entryDatumBis.grid(column=3, row=2, sticky=tk.NW, padx=5, pady=2)
        entryDatumBis.bind('<FocusOut>',self.entry_DatumBis_in_datensatz)

        # option menu
        option_menu_zeitlich = ttk.OptionMenu(self,self.option_var_zeitlich,self.zeitlich[0],*self.zeitlich)
        option_menu_zeitlich.grid(column=4, row=2, sticky=tk.W, padx=5, pady=5)

        # Entry Hinweis Felder unterhalb
        labelDatumVon = ttk.Label(self, text="DD.MM.JJJJ",font=("Verdana", fontsizeEntryText))
        labelDatumVon.grid(column=2, row=3, sticky=tk.NW, padx=5, pady=0)

        labelDatumBis = ttk.Label(self, text="DD.MM.JJJJ",font=("Verdana", fontsizeEntryText))
        labelDatumBis.grid(column=3, row=3, sticky=tk.NW, padx=5, pady=0)

         # Button Zurück
        back_button = ttk.Button(self, text="Go Back", command=lambda: app.clear_frame(StartPage))
        back_button.grid(column=1, row=4, sticky=tk.NW, padx=5, pady=5, columnspan=2)

         # Button Speichern
        new_speichern = ttk.Button(self, text="Save", command=self.save_values)
        new_speichern.grid(column=4, row=4, sticky=tk.N, padx=5, pady=5)
        
    def date_to_sql(self,date_val):
        """Convert date values for the database (YYYY.MM.DD). At the same time we check whether the date values are valid,
        and output an error message if necessary."""
        try: 
            datetime_date = datetime.strptime(date_val, '%d.%m.%Y').date()
        except ValueError: 
            showerror(title="Invalid date", message="Please check your date input!")
        else:
            return datetime_date.strftime('%Y-%m-%d')            
        
    def entry_text_in_datensatz(self,_event):
        self.datensatz[0]=self.entryText_var.get()

    def entry_DatumVon_in_datensatz(self,_event):
        if self.entryDatumVon_var.get()=='':
            self.datensatz[1]=''
        else:
            self.datensatz[1]=self.date_to_sql(self.entryDatumVon_var.get())

    def entry_DatumBis_in_datensatz(self,_event):
        if self.entryDatumBis_var.get()=='':
            self.datensatz[2]=''
        else:
            self.datensatz[2]=self.date_to_sql(self.entryDatumBis_var.get())

    def save_values(self):
        """ Get data field values from formualar and save via habit class.
            Check date values for validity again beforehand. This is necessary, because so far we have only checked after leaving the
            of the entry field. If the user saves without correction, a wrong date value or None would be written into the 
            DB. We also check if a text has been entered."""
        try: 
            datetime_date = datetime.strptime(self.datensatz[1], '%Y-%m-%d').date()
            datetime_date = datetime.strptime(self.datensatz[2], '%Y-%m-%d').date()
        except TypeError: 
            showerror(title="Invalid date", message="Please check your date input!")
        else:       
            if self.datensatz[0]=='':
                showerror(title="Missing values!", message="Values are missing in the data fields!")
            else:
                self.datensatz[3]=self.option_var_zeitlich.get()[0]
                if Habit.selected_row==0:
                    # Neues Habit anlegen
                    Habit.new(self.datensatz[1],self.datensatz[2],self.datensatz[0],self.datensatz[3])
                else:
                    # Vorhandenes Habit updaten
                    habit_shows[Habit.selected_row].update(self.datensatz[0],self.datensatz[1],self.datensatz[2],self.datensatz[3])
                app.clear_frame(StartPage)

    def import_values(self,n):
        """ Enters the stored values into the data fields so that they can be edited. """

        Habit.selected_row=n # Habit variable set. save_values then knows whether to resave or update.
        if n==0:
            # If 0 is passed, then the New button was clicked. The fields are overwritten with '', because they are otherwise filled
            # if the user was already once on the EditNew Seitem.
            self.entryText_var.set('')
            self.entryDatumVon_var.set('')
            self.entryDatumBis_var.set('')
            self.option_var_zeitlich.set(self.zeitlich[0])
        else:
            # If the passed value is greater than 0, it will be the first value in the habit_shows list and will be used to call the habit object.
            self.entryText_var.set(habit_shows[n].name)
            self.datensatz[0]=habit_shows[n].name
            year, month, day = map(str,(habit_shows[n].startdate).split("-"))
            self.entryDatumVon_var.set(day+'.'+month+'.'+year) #strftime('%d.%m.%Y')
            self.datensatz[1]=self.date_to_sql(self.entryDatumVon_var.get())
            year, month, day = map(str,(habit_shows[n].enddate).split("-"))
            self.entryDatumBis_var.set(day+'.'+month+'.'+year) #strftime('%d.%m.%Y')
            self.datensatz[2]=self.date_to_sql(self.entryDatumBis_var.get())
            i=0
            for item in self.zeitlich_kurz:
                if self.zeitlich_kurz[i] == habit_shows[n].periode:
                    self.option_var_zeitlich.set(self.zeitlich[i])
                    self.datensatz[3]=self.option_var_zeitlich.get()[0]
                    break
                i+=1            

class PageInfo(tk.Frame):
    '''Information on selected Habit'''
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        # configure the grid
        self.columnconfigure(0, weight=1)
        self.columnconfigure(1, weight=2)
        self.columnconfigure(2, weight=4)
        self.columnconfigure(3, weight=2)
        self.columnconfigure(4, weight=2)
        self.columnconfigure(5, weight=1)
        self.create_widgets(controller)

    def create_widgets(self,controller):
        fontsizeEntryText=10
        # Titelzeile
        self.info_labelTitle = ttk.Label(self, text='Habit information',font=("Verdana", 14))
        self.info_labelTitle.grid(column=1, row=0, sticky=tk.W, padx=5, pady=5, columnspan=4)

        self.info_labelName = ttk.Label(self, text='Name:',  font=("Verdana", fontsizeEntryText))
        self.info_labelName.grid(column=1, row=2, sticky=tk.W, padx=5, pady=5)
        self.info_labelNameVar = ttk.Label(self,  font=("Verdana", fontsizeEntryText))
        self.info_labelNameVar.grid(column=2, row=2, sticky=tk.W, padx=5, pady=5, columnspan=3)

        self.info_labelDateCreate = ttk.Label(self, text="Creation date:", font=("Verdana", fontsizeEntryText))
        self.info_labelDateCreate.grid(column=1, row=3, sticky=tk.W, padx=5, pady=0)
        self.info_labelDateCreateVar = ttk.Label(self,font=("Verdana", fontsizeEntryText))
        self.info_labelDateCreateVar.grid(column=2, row=3, sticky=tk.W, padx=5, pady=0, columnspan=3)

        self.info_labelStartDate = ttk.Label(self, text="Start date:", font=("Verdana", fontsizeEntryText))
        self.info_labelStartDate.grid(column=1, row=4, sticky=tk.W, padx=5, pady=0)
        self.info_labelStartDateVar = ttk.Label(self,font=("Verdana", fontsizeEntryText))
        self.info_labelStartDateVar.grid(column=2, row=4, sticky=tk.W, padx=5, pady=0, columnspan=3)

        self.info_labelEndDate = ttk.Label(self, text="End date:", font=("Verdana", fontsizeEntryText))
        self.info_labelEndDate.grid(column=1, row=5, sticky=tk.W, padx=5, pady=0)
        self.info_labelEndDateVar = ttk.Label(self,font=("Verdana", fontsizeEntryText))
        self.info_labelEndDateVar.grid(column=2, row=5, sticky=tk.W, padx=5, pady=0, columnspan=3)

        self.info_labelPositive = ttk.Label(self, text="Longest series:", font=("Verdana", fontsizeEntryText))
        self.info_labelPositive.grid(column=1, row=6, sticky=tk.W, padx=5, pady=0)
        self.info_labelPositiveVar = ttk.Label(self,font=("Verdana", fontsizeEntryText))
        self.info_labelPositiveVar.grid(column=2, row=6, sticky=tk.W, padx=5, pady=0, columnspan=4)
        self.info_labelPositiveVar1 = ttk.Label(self,font=("Verdana", fontsizeEntryText))
        self.info_labelPositiveVar1.grid(column=2, row=7, sticky=tk.W, padx=5, pady=0, columnspan=4)

        self.info_labelNegativeVar = ttk.Label(self,font=("Verdana", fontsizeEntryText))
        self.info_labelNegativeVar.grid(column=2, row=8, sticky=tk.W, padx=5, pady=0, columnspan=4)
        self.info_labelNegativeVar1 = ttk.Label(self,font=("Verdana", fontsizeEntryText))
        self.info_labelNegativeVar1.grid(column=2, row=9, sticky=tk.W, padx=5, pady=0, columnspan=4)
         # Button Go Back
        back_button = ttk.Button(self, text="Go back", command=lambda: app.clear_frame(StartPage))
        back_button.grid(column=1, row=10, sticky=tk.NW, padx=5, pady=5, columnspan=2)

    def import_info(self,n):
        Habit.selected_info=n
        self.info_labelNameVar.config(text=habit_shows[n].name)
        year, month, day = map(str,((str(habit_shows[n].datecreate)).split(" "))[0].split("-"))
        hh,mm,ss = map(str,((str(habit_shows[n].datecreate)).split(" "))[1].split(":"))
        self.info_labelDateCreateVar.config(text=day+'.'+month+'.'+year+' um '+hh+':'+mm+':'+(ss.split('.'))[0])
        year, month, day = map(str,(str(habit_shows[n].startdate)).split("-"))
        self.info_labelStartDateVar.config(text="%s.%s.%s" % (day,month,year))
        year, month, day = map(str,(str(habit_shows[n].enddate)).split("-"))
        self.info_labelEndDateVar.config(text="%s.%s.%s" % (day,month,year))
        series_val=habit_shows[n].series_check_values()
        text00='Positive series: Not enough data available yet.'
        text10='Negative series: Not enough data available yet.'
        text1=''
        text2=''
        if series_val[0][0]!='':
            year, month, day = map(str,series_val[0][0].split("-"))
            year1, month1, day1 = map(str,series_val[0][1].split("-"))
            text00='You were successful without any gaps from %s.%s.%s' % (day,month,year)+' to %s.%s.%s' % (day1,month1,year1)+'.'
            if series_val[0][3]=='d':
                if series_val[0][2]==1: 
                    text1='That is after all a whole day.' 
                else:  
                    text1='That is a total of '+str(series_val[0][2])+' days.'
            else:
                if series_val[0][2]==1: 
                    text1="After all, that's a whole week." 
                else:  
                    text1="That is a total of "+str(series_val[0][2])+" weeks."
            
        if series_val[1][0]!='': 
            year, month, day = map(str,series_val[1][0].split("-"))
            year1, month1, day1 = map(str,series_val[1][1].split("-"))
            text10="From %s.%s.%s" % (day,month,year)+" to %s.%s.%s" % (day1,month1,year1)+" you didn't make it."
            if series_val[0][3]=='d':
                if series_val[1][2]==1: 
                    text2='That is after all a whole day.' 
                else:  
                    text2='That is a total of '+str(series_val[1][2])+' days.'
            else:
                if series_val[1][2]==1: 
                    text2="After all, that's a whole week." 
                else:  
                    text2="That is a total of "+str(series_val[1][2])+" weeks."
        self.info_labelPositiveVar.config(text=text00)
        self.info_labelNegativeVar.config(text=text10)
        self.info_labelPositiveVar1.config(text=text1)
        self.info_labelNegativeVar1.config(text=text2)

if __name__ == "__main__":
    app = HabitMain()
    app.mainloop()